In [100]:
from time import sleep
import typing as t
from typing import List
from functools import reduce
from collections import defaultdict
from itertools import chain
import numpy as np
import torch
import random
from tabulate import tabulate
from torch import Tensor

In [73]:
MOONS = ['🌑', '🌒', '🌓', '🌔', '🌕', '🌘', '🌗', '🌖', '🌕']

In [74]:

# Function to initialize the board with starting pieces
def initialize_board():
    board = torch.zeros(8, 8, dtype=torch.long)
    x = torch.randint(7, size=(4,))
    board[3, 3] = -1
    board[3, 4] = 1
    board[4, 3] = 1
    board[4, 4] = -1
    return board



In [75]:
torch.randint(7, size=(4,))

tensor([0, 4, 3, 4])

In [76]:
def get_random_board() -> torch.Tensor:
    i, j = random.randint(0, 20), random.randint(0, 20)
    board = torch.zeros(8, 8, dtype=torch.long)
    x1 = torch.randint(7, size=(i,))
    y1 = torch.randint(7, size=(i,))
    x2 = torch.randint(7, size=(j,))
    y2 = torch.randint(7, size=(j,))

    for x, y in zip(x1, y1):
        board[x, y] = 1
    for x, y in zip(x2, y2):
        board[x, y] = -1
    return board

In [77]:
board = initialize_board()

In [78]:
board

tensor([[ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0, -1,  1,  0,  0,  0],
        [ 0,  0,  0,  1, -1,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0],
        [ 0,  0,  0,  0,  0,  0,  0,  0]])

In [79]:
def get_player_pieces(board: torch.Tensor, side: t.Union[int, t.Tuple[int,...]] = 1) -> torch.Tensor:
    """ Function to select squares by the value assigned to them.
    E.g. select squares where blacks stay. """

    if type(side) == int:
        moves = torch.vstack(torch.where(board == side))

    elif type(side) in (list, tuple):
        # Apply each value step by step
        conditions = [board == s for s in side]

        # Combine all conditions to one
        conditions = reduce(torch.logical_or, conditions)
        moves = torch.vstack(torch.where(conditions))
    else: raise TypeError

    return moves.T  # By trasponing the matrix, we turn [(x1, x2), (y1, y2)] into [(x1, y1),

In [80]:
get_player_pieces(board, side=(1)).tolist()

[[3, 4], [4, 3]]

In [81]:
board[(2,3)] = 1
board[(3,3)] = 0

In [82]:
def get_all_possible_moves(board: torch.Tensor):
    """ We will get a list of all 'valid' the cells that we could move to. Conditions:
    1) These cells are not occupied by any player;
    2) At least ONE of Neighboring cells must be occupied
    (as we cannot place a chip on the edge of the board at the beginning of the game);
    3) Squares are within the borders of the board
    """
    valid_moves = set()
    # Below we transform Tensors to list of tuples to make it strictly deterministic
    # (two tensors with same value might have different hashes)
    all_occupied_squares = get_player_pieces(board, side=(1,-1)).tolist()
    all_available_squares = get_player_pieces(board, side=0).tolist()  # First condition
    for row, col in all_available_squares:
        if (row, col) in all_occupied_squares:
            continue
        # Second condition - check for neighbor squares
        neighbors = torch.tensor(
            [
                [row - 1, col - 1], [row - 1, col], [row - 1, col + 1],
                [row, col - 1],                         [row, col + 1],
                [row + 1, col - 1], [row + 1, col], [row + 1, col + 1],

            ]
        )

        # Check for Condition №3
        neighbors = (
            neighbors[
                (neighbors[:, 0] >= 0) &
                (neighbors[:, 0] < 8) &
                (neighbors[:, 1] >= 0) &
                (neighbors[:, 1] < 8)
            ]
        ).tolist()
        if any([cell in all_occupied_squares for cell in neighbors]):
            valid_moves.add((row, col))
    return sorted(tuple(valid_moves))

In [278]:
def check_line_values(first, second):
    # if second in (first,):
    #     return first
    # else:
    #     return False
    print(first, second)
    return first if first == second else False

In [279]:
bool(reduce(check_line_values, [-11, -1, 1][1:-1]))

True

In [257]:
def is_good_list(lst):
    print(lst)
    if len(lst) < 3:
        return False

    if lst[0] != lst[-1]:
        return False

    return bool(reduce(check_line_values, lst[1:-1]))
    for i in range(1, len(lst) - 1):
        if lst[i] == lst[0] or lst[i] == lst[-1]:
            return False

    assert bool(reduce(check_line_values, lst[1:-1]))
    return True

In [258]:
def validate_move(move: t.Tuple[int, int],
                  board: torch.Tensor,
                  player: int = 1
  ) -> bool:
    """
    Function to validate moves. We use separated function to be able to use generator
    """
    row_directions = [-1, -1, -1,  0,  0, +1, +1, +1]
    col_directions = [-1,  0, +1, -1, +1, -1,  0, +1]
    x, y = move
    for dx, dy in zip(row_directions, col_directions):
        line = [player]
        while True:
            x1 = x + dx
            y1 = y + dy
            if not (
                    (0 <= x1 < 8) and (0 <= y1 < 8)
            ):
                line.append(None)
                break
            next_square = board[x1, y1].item()

            if next_square in (0, None):
                print("LINE: ", line, " OUT")
                break
            else:
                # next_square is -1 or 1:
                line.append(next_square)

            if dx > 0:
                dx += 1
            elif dx < 0:
                dx -= 1
            if dy > 0:
                dy += 1
            elif dy < 0:
                dy -= 1
        if is_good_list(line):
            return True

    return False

In [259]:
def get_captures(move: t.Tuple[int, int],
                  board: torch.Tensor,
                  player: int = 1
                  ) -> bool:
    """
    Function to validate moves. We use separated function to be able to use generator
    """
    row_directions = [-1, -1, -1,  0,  0, +1, +1, +1]
    col_directions = [-1,  0, +1, -1, +1, -1,  0, +1]
    x, y = move
    captures = []
    for dx, dy in zip(row_directions, col_directions):
        line = [player]
        capture = []
        while True:
            x1 = x + dx
            y1 = y + dy
            if not (
                    (0 <= x1 < 8) and (0 <= y1 < 8)
            ):
                line.append(0)
                break
            next_square = board[x1, y1].item()

            if next_square == 0:
                break
            else:
                # next_square is -1 or 1:
                if next_square == -1 * player:
                    capture.append((x1, y1))
                line.append(next_square)


            if dx > 0:
                dx += 1
            elif dx < 0:
                dx -= 1
            if dy > 0:
                dy += 1
            elif dy < 0:
                dy -= 1
        if is_good_list(line):
            captures.append(capture)
        # else:
        #     print("BAD LINE: ", line)
    return captures

In [260]:
def get_valid_moves(board: torch.Tensor, moves: torch.Tensor, player = 1) -> torch.Tensor:
    """
    After we got list of possible moves, we validate them to get the
    list of moves which we are actually allowed to do.
    That is where the basic rule of the Reversi game takes effect.
    """

    valid_moves = set()
    print(moves)

    [valid_moves.add(move) for move in moves if validate_move(move, board, player)]
    print(valid_moves)
    return valid_moves

In [261]:
def show_board(board: torch.Tensor, first_player_pieces, second_player_pieces, player=1):
    # board  = board.detach().clone().numpy().astype(str)
    # print(torch.gather(board, index=torch.where((board[:, 0] >= 0) & (board[:, 0] < 8))))

    available_squares = get_all_possible_moves(board)
    available_squares = get_valid_moves(board, available_squares, player)
    board = np.full((8, 8), fill_value='⬜️', dtype=str)
    for p in first_player_pieces:
        board[*p] = '🌑'
    for p in second_player_pieces:
        board[*p] = '🌕'

    for p in available_squares:
        board[*p] = '🟪'

    board = tabulate(board, showindex=True, headers=["/", "a", "b", "c", "d", "e", "f", "g", "h"],
                                  tablefmt="simple"
    )
    print(board)

In [262]:
board = initialize_board()

In [263]:
show_board(
    board,
    get_player_pieces(board, side=(1,)),
    get_player_pieces(board, side=(-1)),
    player=1
)

[(2, 2), (2, 3), (2, 4), (2, 5), (3, 2), (3, 5), (4, 2), (4, 5), (5, 2), (5, 3), (5, 4), (5, 5)]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1, -1, -1]  OUT
[1, -1, -1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1, -1, 1]  OUT
[1, -1, 1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1, -1]  OUT
[1, -1]
LINE:  [1, 1, -1]  OUT
[1, 1, -1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1, 1, 1]  OUT
[1, 1, 1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1, -1, 1]  OUT
[1, -1, 1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1]  OUT
[1]
LINE:  [1, 1, -1]  OUT
[1, 1, -1]
LINE:  [1]  OUT
[1]
LINE:  [1, -1]  OUT
[1, -1]
LIN

In [208]:
# board = get_random_board()
board = initialize_board()
player = 1
i = 0
while i < 100:
    show_board(
        board,
        get_player_pieces(board, side=(1,)),
        get_player_pieces(board, side=(-1)),
        player=player
        )
    moves = get_valid_moves(board, get_all_possible_moves(board), player)
    if moves:
        # If no moves, player has to pass
        move = random.choice(tuple(moves))
        board[move] = player
        captures = get_captures(move, board, player)
        captures = list(chain.from_iterable(captures))
        for capture in captures:
            board[capture] = player
    else:
        # print(set(get_all_possible_moves(board)) & get_valid_moves(board, get_all_possible_moves(board), player))
        print(f"Player {player} passes!")
        break
    player *= -1
    i += 1

    if not torch.any(board == 0):
        print("GAME OVER")


[(2, 2), (2, 3), (2, 4), (2, 5), (3, 2), (3, 5), (4, 2), (4, 5), (5, 2), (5, 3), (5, 4), (5, 5)]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1, -1, -1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1, -1]
BAD LINE:  [1, 1, -1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1, 1, 1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1, 1, -1]
BAD LINE:  [1]
BAD LINE:  [1, -1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1, -1]
BAD LINE:  [1]
BAD LINE:  [1, 1, -1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1, 1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1]
BAD LINE:  [1, 1, 1]
BAD LIN

In [178]:
{1, 3,4} & {1, 5}

{1}

In [129]:
bool(set())

False

In [113]:
nested_list = [[(1, 2), (2, 4), 3], [4, (5, 1), 6], [7, 8, 9]]

In [115]:
# %%timeit
basic_list = [item for sublist in nested_list for item in sublist]
basic_list

[(1, 2), (2, 4), 3, 4, (5, 1), 6, 7, 8, 9]

In [116]:
# %%timeit
basic_list = np.array(nested_list).flatten()
basic_list

ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 2 dimensions. The detected shape was (3, 3) + inhomogeneous part.

In [117]:
# %%timeit
basic_list = []
for lst in nested_list:
    for item in lst:
        basic_list.append(item)
basic_list

[(1, 2), (2, 4), 3, 4, (5, 1), 6, 7, 8, 9]

In [118]:
# %%timeit
basic_list = list(chain(*nested_list))
basic_list

[(1, 2), (2, 4), 3, 4, (5, 1), 6, 7, 8, 9]

In [120]:
# %%timeit
basic_list = list(chain.from_iterable(nested_list))
basic_list

[(1, 2), (2, 4), 3, 4, (5, 1), 6, 7, 8, 9]

In [47]:
def vvvvvalidate_move(move: t.Tuple[int, int], board: torch.Tensor):
    row, col = move
    print(move, board[move].item())

    t = torch.tensor(
        [

            [row + 1, col], [row - 1, col],


            [row, col + 1], [row, col - 1],

        ]
    )
    print(t)
    # t = t[t < 8]

    # Exclude the moves that goes beyond the borders
    t = t[(t[:, 0] < 8) & (t[:, 1] < 8)]

    print(t)
    # print(torch.split(t, 2))
    # print(t, t[t < 8].reshape(-1, 2), sep='\n')
    # print(board[t])
    # print(
    #     torch.any(
    #         t < 8, dim=-1,
    #     ).shape
    # )
    # print(board[move].item())

In [19]:
print(get_player_pieces(board, side=0))
torch.tensor([7, 1]) in get_player_pieces(board, side=0)

tensor([[0, 3],
        [0, 4],
        [0, 6],
        [0, 7],
        [1, 0],
        [1, 1],
        [1, 3],
        [1, 5],
        [1, 6],
        [1, 7],
        [2, 0],
        [2, 1],
        [2, 4],
        [2, 6],
        [2, 7],
        [3, 1],
        [3, 3],
        [3, 5],
        [3, 6],
        [3, 7],
        [4, 0],
        [4, 3],
        [4, 4],
        [4, 6],
        [4, 7],
        [5, 0],
        [5, 1],
        [5, 2],
        [5, 4],
        [5, 5],
        [5, 6],
        [5, 7],
        [6, 0],
        [6, 7],
        [7, 0],
        [7, 1],
        [7, 2],
        [7, 3],
        [7, 4],
        [7, 5],
        [7, 6],
        [7, 7]])


True

In [20]:


# Function to make a move
def make_move(board, row, col, player):
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]
    opponent = -1 if player == 1 else 1
    # validate_move((row, col), board)
    # pieces = get_player_pieces(board, side=0)
    # print(f"CHECK: {torch.tensor((row, col)) in pieces}")
    board[row, col] = player
    for dx, dy in directions:
        x, y = row + dx, col + dy
        flipped = []
        while 0 <= x < 8 and 0 <= y < 8 and board[x, y] == opponent:
            flipped.append((x, y))
            x += dx
            y += dy
        if 0 <= x < 8 and 0 <= y < 8 and board[x, y] == player:
            for fx, fy in flipped:
                board[fx, fy] = player
    return board

In [21]:

# Function to check if the game is over
def is_game_over(board):
    return not torch.any(board == 0)

In [22]:

# Main game loop
def play_game():
    board = initialize_board()
    # print(board)
    show_board(board, get_player_pieces(board, side=(1,)), get_player_pieces(board, side=(-1)) )
    player = 1
    while not is_game_over(board):
        # print_board(board)
        show_board(board, get_player_pieces(board, side=(1,)), get_player_pieces(board, side=(-1)) )
        if player == 1:
            valid_moves = []
            for i in range(8):
                for j in range(8):
                    if board[i, j] == 0:
                        valid_moves.append((i, j))
            if valid_moves:
                row, col = random.choice(valid_moves)
                board = make_move(board, row, col, player)
        else:
            print("Computer's turn...")
            valid_moves = []
            for i in range(8):
                for j in range(8):
                    if board[i, j] == 0:
                        valid_moves.append((i, j))
            if valid_moves:
                row, col = random.choice(valid_moves)
                board = make_move(board, row, col, player)
        player = -1 if player == 1 else 1

    show_board(board, get_player_pieces(board, side=(1,)), get_player_pieces(board, side=(-1)) )
    x_count = torch.sum(board == 1)
    o_count = torch.sum(board == -1)
    if x_count > o_count:
        print("Player X wins!")
    elif o_count > x_count:
        print("Player O wins!")
    else:
        print("It's a tie!")

In [23]:

# Run the game
play_game()

TypeError: validate_move() takes 2 positional arguments but 3 were given